# Model for Microsoft
Sentiment is built upon the Reuters titles dataset.
Historical data is taken from yahoo finance

In [1]:
from tqdm import tqdm
import numpy as np
import pickle
import nltk
import string
import os
import pandas as pd
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import re
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import time
import sys
import time
from tqdm._tqdm_notebook import tqdm_notebook
from keras.models import Sequential, load_model
from keras import layers
from keras.optimizers import RMSprop
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import logging
from datetime import datetime, timedelta
from io import StringIO
import copy

C:\Users\48570\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
Using TensorFlow backend.


# Scraping historical data from yahoo finance

In [2]:
class YahooFinanceHistory:
    timeout = 2
    crumb_link = 'https://finance.yahoo.com/quote/{0}/history?p={0}'
    crumble_regex = r'CrumbStore":{"crumb":"(.*?)"}'
    quote_link = 'https://query1.finance.yahoo.com/v7/finance/download/{quote}?period1={dfrom}&period2={dto}&interval=1d&events=history&crumb={crumb}'

    def __init__(self, symbol, days_back=7):
        self.symbol = symbol
        self.session = requests.Session()
        self.dt = timedelta(days=days_back)

#requesting crumb and cookie
    def get_crumb(self):
        response = self.session.get(self.crumb_link.format(self.symbol), timeout=self.timeout)
        response.raise_for_status()
        match = re.search(self.crumble_regex, response.text)
        if not match:
            raise ValueError('Could not get crumb from Yahoo Finance')
        else:
            self.crumb = match.group(1)

#requesting data
    def get_quote(self):
        if not hasattr(self, 'crumb') or len(self.session.cookies) == 0:
            self.get_crumb()
        now = datetime.utcnow()
        dateto = int(now.timestamp())
        datefrom = int((now - self.dt).timestamp())
        url = self.quote_link.format(quote=self.symbol, dfrom=datefrom, dto=dateto, crumb=self.crumb)
        response = self.session.get(url)
        response.raise_for_status()
        return pd.read_csv(StringIO(response.text), parse_dates=['Date'])

In [3]:
# extracting data about Microsoft from 400 days back
df_v = YahooFinanceHistory('MSFT', days_back=4000).get_quote()

In [4]:
#sorting dates from the latest to earliest
df_v.sort_values(by='Date')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-06-04,21.770000,21.900000,21.580000,21.830000,16.964777,42330000
1,2009-06-05,21.959999,22.309999,21.809999,22.139999,17.205687,59579200
2,2009-06-08,21.980000,22.320000,21.629999,22.049999,17.135746,49000600
3,2009-06-09,22.059999,22.320000,21.879999,22.080000,17.159063,50887700
4,2009-06-10,22.170000,22.620001,22.120001,22.549999,17.524307,61297200
...,...,...,...,...,...,...,...
2752,2020-05-11,183.149994,187.509995,182.850006,186.740005,186.740005,30809400
2753,2020-05-12,186.800003,187.039993,182.300003,182.509995,182.509995,32038200
2754,2020-05-13,182.550003,184.050003,176.539993,179.750000,179.750000,44711500
2755,2020-05-14,177.539993,180.690002,175.679993,180.529999,180.529999,41873900


In [5]:
df_v.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object

In [6]:
df_v

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-06-04,21.770000,21.900000,21.580000,21.830000,16.964777,42330000
1,2009-06-05,21.959999,22.309999,21.809999,22.139999,17.205687,59579200
2,2009-06-08,21.980000,22.320000,21.629999,22.049999,17.135746,49000600
3,2009-06-09,22.059999,22.320000,21.879999,22.080000,17.159063,50887700
4,2009-06-10,22.170000,22.620001,22.120001,22.549999,17.524307,61297200
...,...,...,...,...,...,...,...
2752,2020-05-11,183.149994,187.509995,182.850006,186.740005,186.740005,30809400
2753,2020-05-12,186.800003,187.039993,182.300003,182.509995,182.509995,32038200
2754,2020-05-13,182.550003,184.050003,176.539993,179.750000,179.750000,44711500
2755,2020-05-14,177.539993,180.690002,175.679993,180.529999,180.529999,41873900


## Sentiment for all the articles with "Microsoft" in the body of an article

In [7]:
# uploading file
df2 = pd.read_csv('df_MS.csv')

In [8]:
df2

,Unnamed: 0,Date,compound_mean
0,0,2011-07-06,0.075921
1,1,2011-07-07,0.765000
2,2,2011-07-08,0.427933
3,3,2011-07-09,0.075921
4,4,2011-07-11,0.446600
...,...,...,...
820,820,2016-12-05,0.075921
821,821,2016-12-06,0.670500
822,822,2016-12-13,0.381800
823,823,2016-12-20,0.571900


In [9]:
# column Unnamed isn't needed
df2 = df2.drop(['Unnamed: 0'], axis=1)

In [10]:
df2

,Date,compound_mean
0,2011-07-06,0.075921
1,2011-07-07,0.765000
2,2011-07-08,0.427933
3,2011-07-09,0.075921
4,2011-07-11,0.446600
...,...,...
820,2016-12-05,0.075921
821,2016-12-06,0.670500
822,2016-12-13,0.381800
823,2016-12-20,0.571900


In [11]:
df2.dtypes

Date              object
compound_mean    float64
dtype: object

In [12]:
#changing column Date type to datetime type
df2.Date=pd.to_datetime(df2['Date'])

In [13]:
#merging dataframe with historical data with dataframe with sentiments 
df3 = pd.merge(df_v,df2,on='Date')

In [14]:
df3

,Date,Open,High,Low,Close,Adj Close,Volume,compound_mean
0,2011-07-06,25.969999,26.370001,25.959999,26.330000,21.362141,48744200,0.075921
1,2011-07-07,26.490000,26.879999,26.360001,26.770000,21.719128,51946500,0.765000
2,2011-07-08,26.540001,26.980000,26.510000,26.920000,21.840824,58320700,0.427933
3,2011-07-11,26.620001,26.799999,26.490000,26.629999,21.605539,43999800,0.446600
4,2011-07-12,26.549999,26.790001,26.340000,26.540001,21.532520,47319300,0.440400
...,...,...,...,...,...,...,...,...
747,2016-12-05,59.700001,60.590000,59.560001,60.220001,56.903275,23552700,0.075921
748,2016-12-06,60.430000,60.459999,59.799999,59.950001,56.648148,19907000,0.670500
749,2016-12-13,62.500000,63.419998,62.240002,62.980000,59.511261,35718900,0.381800
750,2016-12-20,63.689999,63.799999,63.029999,63.540001,60.040417,26028400,0.571900


# Machine learning for prediction of label for the next day 

In [15]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import sklearn

In [16]:
#deepcopying dataframe, so there would be no need to run everything from the beggining
df = copy.deepcopy(df3)

In [17]:
#label indicates wheter the price will go up(1) or down(0) next day
def add_label(dfi):
    idx = len(dfi.columns)
    new_col = np.where(dfi['Close'] >= dfi['Close'].shift(1), 1, 0)  
    dfi.insert(loc=idx, column='Label', value=new_col)
    dfi = dfi.fillna(0)
    df['Label'] =  df['Label'].shift(-1, axis = 0)
    df.drop(df.index[len(df)-1], inplace = True)

In [19]:
add_label(df)

In [20]:
df.dtypes

Date             datetime64[ns]
Open                    float64
High                    float64
Low                     float64
Close                   float64
Adj Close               float64
Volume                    int64
compound_mean           float64
Label                   float64
dtype: object

In [23]:
array = df.values

In [24]:
# train and test
X = array[:,1:8]
Y = array[:,8]

In [25]:
X = sklearn.preprocessing.MinMaxScaler().fit_transform(X)

In [26]:
Y = Y.astype('int')

In [27]:
print(X[0])
print(df.columns[1:9])

[0.04891574 0.04782501 0.05530067 0.05940344 0.04408596 0.16596441
 0.52541799]
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'compound_mean',
       'Label'],
      dtype='object')


In [28]:
# choosing best features for the model
test = SelectKBest(score_func=chi2, k=5)
fit = test.fit(X, Y)
np.set_printoptions(precision=3)
print(fit.scores_)
features = fit.transform(X)
print(features[0:5,:])

[0.06  0.06  0.066 0.066 0.04  0.001 0.051]
[[0.049 0.048 0.055 0.059 0.525]
 [0.062 0.061 0.065 0.071 0.938]
 [0.063 0.063 0.069 0.074 0.736]
 [0.065 0.059 0.069 0.067 0.747]
 [0.064 0.059 0.065 0.065 0.744]]


In [29]:
#Open, High, Low, Close and compound mean give the most information
features

array([[0.049, 0.048, 0.055, 0.059, 0.525],
       [0.062, 0.061, 0.065, 0.071, 0.938],
       [0.063, 0.063, 0.069, 0.074, 0.736],
       ...,
       [0.918, 0.915, 0.918, 0.909, 0.881],
       [0.97 , 0.99 , 0.98 , 0.986, 0.708],
       [1.   , 1.   , 1.   , 1.   , 0.822]])

In [30]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split

#building and training a model
X_train, X_test, y_train, y_test = train_test_split(features, Y, test_size=0.25)
model = Sequential()
model.add(Dense(16, input_shape=(5,) ))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                       patience=100, min_delta=0.0001, restore_best_weights = True)
history = model.fit(X_train, y_train, epochs=100,
                    validation_data= (X_test,y_test),
                              callbacks=[es])

Train on 563 samples, validate on 188 samples
Epoch 1/100
563/563 [==============================] - ETA: 23s - loss: 0.7019 - accuracy: 0.500 - ETA: 1s - loss: 0.6953 - accuracy: 0.486 - ETA: 0s - loss: 0.6929 - accuracy: 0.50 - 2s 3ms/step - loss: 0.6935 - accuracy: 0.4920 - val_loss: 0.6971 - val_accuracy: 0.4734
Epoch 2/100
563/563 [==============================] - ETA: 0s - loss: 0.6782 - accuracy: 0.62 - ETA: 0s - loss: 0.6934 - accuracy: 0.46 - ETA: 0s - loss: 0.6968 - accuracy: 0.48 - 0s 294us/step - loss: 0.6959 - accuracy: 0.4938 - val_loss: 0.6920 - val_accuracy: 0.5266
Epoch 3/100
563/563 [==============================] - ETA: 0s - loss: 0.7037 - accuracy: 0.43 - ETA: 0s - loss: 0.6974 - accuracy: 0.49 - ETA: 0s - loss: 0.6974 - accuracy: 0.50 - 0s 269us/step - loss: 0.6988 - accuracy: 0.5080 - val_loss: 0.6919 - val_accuracy: 0.5266
Epoch 4/100
563/563 [==============================] - ETA: 0s - loss: 0.6783 - accuracy: 0.65 - ETA: 0s - loss: 0.6954 - accuracy: 0.53 - E

Epoch 55/100
563/563 [==============================] - ETA: 0s - loss: 0.6985 - accuracy: 0.50 - ETA: 0s - loss: 0.6925 - accuracy: 0.52 - ETA: 0s - loss: 0.6933 - accuracy: 0.53 - ETA: 0s - loss: 0.6928 - accuracy: 0.53 - ETA: 0s - loss: 0.6924 - accuracy: 0.53 - 0s 498us/step - loss: 0.6915 - accuracy: 0.5400 - val_loss: 0.6944 - val_accuracy: 0.4734
Epoch 56/100
563/563 [==============================] - ETA: 0s - loss: 0.6929 - accuracy: 0.53 - ETA: 0s - loss: 0.6894 - accuracy: 0.55 - ETA: 0s - loss: 0.6906 - accuracy: 0.52 - ETA: 0s - loss: 0.6920 - accuracy: 0.51 - ETA: 0s - loss: 0.6932 - accuracy: 0.50 - 0s 524us/step - loss: 0.6939 - accuracy: 0.4973 - val_loss: 0.6952 - val_accuracy: 0.4681
Epoch 57/100
563/563 [==============================] - ETA: 0s - loss: 0.6763 - accuracy: 0.59 - ETA: 0s - loss: 0.6945 - accuracy: 0.46 - ETA: 0s - loss: 0.6897 - accuracy: 0.52 - ETA: 0s - loss: 0.6878 - accuracy: 0.53 - ETA: 0s - loss: 0.6900 - accuracy: 0.53 - ETA: 0s - loss: 0.6893

Epoch 79/100
563/563 [==============================] - ETA: 0s - loss: 0.6852 - accuracy: 0.50 - ETA: 0s - loss: 0.6910 - accuracy: 0.56 - ETA: 0s - loss: 0.6903 - accuracy: 0.56 - ETA: 0s - loss: 0.6912 - accuracy: 0.54 - ETA: 0s - loss: 0.6903 - accuracy: 0.55 - 0s 693us/step - loss: 0.6909 - accuracy: 0.5506 - val_loss: 0.6944 - val_accuracy: 0.4840
Epoch 80/100
563/563 [==============================] - ETA: 0s - loss: 0.6868 - accuracy: 0.53 - ETA: 0s - loss: 0.6869 - accuracy: 0.51 - ETA: 0s - loss: 0.6897 - accuracy: 0.50 - ETA: 0s - loss: 0.6864 - accuracy: 0.51 - ETA: 0s - loss: 0.6893 - accuracy: 0.50 - 0s 521us/step - loss: 0.6899 - accuracy: 0.5027 - val_loss: 0.6950 - val_accuracy: 0.4840
Epoch 81/100
563/563 [==============================] - ETA: 0s - loss: 0.6794 - accuracy: 0.56 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6949 - accuracy: 0.50 - ETA: 0s - loss: 0.6937 - accuracy: 0.50 - 0s 397us/step - loss: 0.6940 - accuracy: 0.5044 - val_loss: 0.69

In [31]:
#evaluating the model
model.evaluate(X_test, y_test)

188/188 [==============================] - ETA:  - 0s 149us/step


[0.6936830586575448, 0.4680851101875305]

# LSTM AND GRU METHOD

In [46]:
normalized_df1 = copy.deepcopy(df3)

In [47]:
normalized_df1 = normalized_df1[['Open','High','Low','Close','compound_mean']]

In [48]:
# normalizing function
def normalized_df(df):
    normalized_df=(df-df.mean())/df.std()
    return normalized_df

In [49]:
#for later
normalized_df2 = copy.deepcopy(normalized_df1)

In [50]:
normalized_df2

,Open,High,Low,Close,compound_mean
0,25.969999,26.370001,25.959999,26.330000,0.075921
1,26.490000,26.879999,26.360001,26.770000,0.765000
2,26.540001,26.980000,26.510000,26.920000,0.427933
3,26.620001,26.799999,26.490000,26.629999,0.446600
4,26.549999,26.790001,26.340000,26.540001,0.440400
...,...,...,...,...,...
747,59.700001,60.590000,59.560001,60.220001,0.075921
748,60.430000,60.459999,59.799999,59.950001,0.670500
749,62.500000,63.419998,62.240002,62.980000,0.381800
750,63.689999,63.799999,63.029999,63.540001,0.571900


In [51]:
mean = normalized_df1.mean(axis = 0)
normalized_df1 -= mean
std = normalized_df1.std(axis=0)
normalized_df1 /= std

In [52]:
# adding label: up or down or steady
def add_label(df):
    idx = len(df.columns)
    new_col = np.where(df['Close'] >= df['Close'].shift(1), 1, 0)  
    df.insert(loc=idx, column='Label', value=new_col)
    df = df.fillna(0)

In [53]:
add_label(normalized_df1)

In [54]:
normalized_df1 = normalized_df1.values

In [55]:
normalized_df1

array([[-1.234, -1.22 , -1.208, -1.198, -0.012,  0.   ],
       [-1.181, -1.169, -1.167, -1.154,  2.354,  1.   ],
       [-1.176, -1.159, -1.152, -1.139,  1.197,  1.   ],
       ...,
       [ 2.43 ,  2.473,  2.452,  2.472,  1.038,  1.   ],
       [ 2.549,  2.511,  2.531,  2.528,  1.691,  1.   ],
       [ 2.442,  2.418,  2.463,  2.444,  1.846,  0.   ]])

In [56]:
from keras.utils import to_categorical

# defining our generator
def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=32, step=5):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
        samples = np.zeros((len(rows),
                           lookback // step,
                           data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][-1]
        yield samples, to_categorical(targets)

In [57]:
lookback = 30
step = 10
delay = 1
batch_size = 32

In [58]:
#Splitting data into train, test and validation set
train_gen = generator(normalized_df1,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=round(0.6*len(normalized_df1)),
                      shuffle=False,
                      step=step,
                      batch_size=batch_size)
val_gen = generator(normalized_df1,
                    lookback=lookback,
                    delay=delay,
                    min_index=round(0.6*len(normalized_df1))+1,
                    max_index=round(0.8*len(normalized_df1)),
                    step=step,
                    batch_size=batch_size)
test_gen = generator(normalized_df1,
                     lookback=lookback,
                     delay=delay,
                     min_index=round(0.8*len(normalized_df1))+1,
                     max_index=None,
                     step=step,
                     batch_size=batch_size)

val_steps = (round(0.8*len(normalized_df1)) - round(0.6*len(normalized_df1))+1 - lookback) # how many steps to draw from val_gen in order to see the entire validation set
test_steps = (len(normalized_df1) - round(0.8*len(normalized_df1))+1 - lookback)
# How many steps to draw from test_gen in order to see the entire test set

In [59]:
#Building and training LSTM model
model = Sequential()
model.add(LSTM(100, return_sequences=True,
                    input_shape=(None, normalized_df1.shape[-1]),
                    kernel_initializer='random_uniform'))
model.add(Dropout(0.4))
model.add(LSTM(60, dropout=0.0, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(20,activation='relu'))
model.add(layers.Dense(2, activation='softmax'))
model.compile(loss='mean_squared_error', optimizer=RMSprop(),metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                       patience=100, min_delta=0.0001, restore_best_weights = True)

history = model.fit_generator(train_gen,
                              steps_per_epoch=2,
                              epochs=250,
                              validation_data=val_gen,
                              validation_steps=val_steps,
                              callbacks=[es])

Epoch 1/250
2/2 [==============================] - ETA: 2s - loss: 0.2497 - accuracy: 0.56 - 3s 2s/step - loss: 0.2484 - accuracy: 0.5625 - val_loss: 0.2511 - val_accuracy: 0.4380
Epoch 2/250
2/2 [==============================] - ETA: 0s - loss: 0.2511 - accuracy: 0.50 - 1s 267ms/step - loss: 0.2522 - accuracy: 0.4375 - val_loss: 0.2498 - val_accuracy: 0.4375
Epoch 3/250
2/2 [==============================] - ETA: 0s - loss: 0.2488 - accuracy: 0.68 - 0s 226ms/step - loss: 0.2499 - accuracy: 0.5781 - val_loss: 0.2499 - val_accuracy: 0.4267
Epoch 4/250
2/2 [==============================] - ETA: 0s - loss: 0.2491 - accuracy: 0.56 - 1s 328ms/step - loss: 0.2496 - accuracy: 0.5312 - val_loss: 0.2507 - val_accuracy: 0.4380
Epoch 5/250
2/2 [==============================] - ETA: 0s - loss: 0.2482 - accuracy: 0.53 - 0s 223ms/step - loss: 0.2483 - accuracy: 0.5469 - val_loss: 0.2499 - val_accuracy: 0.4375
Epoch 6/250
2/2 [==============================] - ETA: 0s - loss: 0.2494 - accuracy: 0.

Epoch 46/250
2/2 [==============================] - ETA: 0s - loss: 0.2503 - accuracy: 0.46 - 1s 290ms/step - loss: 0.2492 - accuracy: 0.5312 - val_loss: 0.2504 - val_accuracy: 0.4380
Epoch 47/250
2/2 [==============================] - ETA: 0s - loss: 0.2475 - accuracy: 0.56 - 1s 355ms/step - loss: 0.2487 - accuracy: 0.5312 - val_loss: 0.2496 - val_accuracy: 0.4375
Epoch 48/250
2/2 [==============================] - ETA: 0s - loss: 0.2555 - accuracy: 0.37 - 1s 275ms/step - loss: 0.2525 - accuracy: 0.4375 - val_loss: 0.2514 - val_accuracy: 0.4370
Epoch 49/250
2/2 [==============================] - ETA: 0s - loss: 0.2518 - accuracy: 0.40 - 1s 352ms/step - loss: 0.2497 - accuracy: 0.4531 - val_loss: 0.2499 - val_accuracy: 0.5013
Epoch 50/250
2/2 [==============================] - ETA: 0s - loss: 0.2490 - accuracy: 0.62 - 1s 333ms/step - loss: 0.2488 - accuracy: 0.5625 - val_loss: 0.2496 - val_accuracy: 0.4375
Epoch 51/250
2/2 [==============================] - ETA: 0s - loss: 0.2481 - acc

Epoch 91/250
2/2 [==============================] - ETA: 0s - loss: 0.2489 - accuracy: 0.59 - 1s 334ms/step - loss: 0.2497 - accuracy: 0.5625 - val_loss: 0.2500 - val_accuracy: 0.4695
Epoch 92/250
2/2 [==============================] - ETA: 0s - loss: 0.2398 - accuracy: 0.59 - 1s 283ms/step - loss: 0.2431 - accuracy: 0.5781 - val_loss: 0.2489 - val_accuracy: 0.4480
Epoch 93/250
2/2 [==============================] - ETA: 0s - loss: 0.2516 - accuracy: 0.50 - 1s 339ms/step - loss: 0.2525 - accuracy: 0.4375 - val_loss: 0.2543 - val_accuracy: 0.4580
Epoch 94/250
2/2 [==============================] - ETA: 0s - loss: 0.2515 - accuracy: 0.43 - 1s 283ms/step - loss: 0.2516 - accuracy: 0.4844 - val_loss: 0.2493 - val_accuracy: 0.5015
Epoch 95/250
2/2 [==============================] - ETA: 0s - loss: 0.2516 - accuracy: 0.46 - 1s 366ms/step - loss: 0.2473 - accuracy: 0.5625 - val_loss: 0.2489 - val_accuracy: 0.4782
Epoch 96/250
2/2 [==============================] - ETA: 0s - loss: 0.2530 - acc

2/2 [==============================] - ETA: 0s - loss: 0.2568 - accuracy: 0.56 - 1s 288ms/step - loss: 0.2551 - accuracy: 0.5000 - val_loss: 0.2470 - val_accuracy: 0.4787
Epoch 180/250
2/2 [==============================] - ETA: 0s - loss: 0.2480 - accuracy: 0.50 - 1s 350ms/step - loss: 0.2501 - accuracy: 0.4531 - val_loss: 0.2694 - val_accuracy: 0.4577
Epoch 181/250
2/2 [==============================] - ETA: 0s - loss: 0.2477 - accuracy: 0.53 - 1s 335ms/step - loss: 0.2463 - accuracy: 0.5469 - val_loss: 0.2518 - val_accuracy: 0.4485
Epoch 182/250
2/2 [==============================] - ETA: 0s - loss: 0.2480 - accuracy: 0.50 - 1s 306ms/step - loss: 0.2477 - accuracy: 0.5469 - val_loss: 0.2467 - val_accuracy: 0.4270
Epoch 183/250
2/2 [==============================] - ETA: 0s - loss: 0.2421 - accuracy: 0.59 - 1s 343ms/step - loss: 0.2443 - accuracy: 0.5781 - val_loss: 0.2743 - val_accuracy: 0.4472
Epoch 184/250
2/2 [==============================] - ETA: 0s - loss: 0.2523 - accuracy: 0

In [60]:
test_loss, test_acc = model.evaluate_generator(test_gen, steps=3)
print('test acc:', test_acc)
print("test_loss:", test_loss)

test acc: 0.5833333134651184
test_loss: 0.2430008053779602


In [61]:
#Building and training a model with GRU
model = Sequential()
model.add(layers.GRU(32,
                     dropout=0.3,
                     recurrent_dropout=0.2,
                     return_sequences=True,
                     input_shape=(None, normalized_df1.shape[-1])))
model.add(layers.GRU(64, activation='relu',
                     dropout=0.3,
                     recurrent_dropout=0.1))
model.add(layers.Dense(2, activation='softmax'))
model.compile(optimizer=RMSprop(), loss='mean_squared_error', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                       patience=50, min_delta=0.0001, restore_best_weights = True)
    
history = model.fit_generator(train_gen,
                              steps_per_epoch=2,
                              epochs=250,
                              validation_data=val_gen,
                              validation_steps=val_steps,
                              callbacks=[es])

Epoch 1/250
2/2 [==============================] - ETA: 3s - loss: 0.2512 - accuracy: 0.43 - 5s 2s/step - loss: 0.2508 - accuracy: 0.4844 - val_loss: 0.2577 - val_accuracy: 0.4370
Epoch 2/250
2/2 [==============================] - ETA: 0s - loss: 0.2479 - accuracy: 0.53 - 1s 364ms/step - loss: 0.2493 - accuracy: 0.5312 - val_loss: 0.2516 - val_accuracy: 0.4380
Epoch 3/250
2/2 [==============================] - ETA: 0s - loss: 0.2534 - accuracy: 0.53 - 1s 371ms/step - loss: 0.2502 - accuracy: 0.5312 - val_loss: 0.2508 - val_accuracy: 0.4375
Epoch 4/250
2/2 [==============================] - ETA: 0s - loss: 0.2457 - accuracy: 0.50 - 1s 283ms/step - loss: 0.2489 - accuracy: 0.5156 - val_loss: 0.2564 - val_accuracy: 0.4370
Epoch 5/250
2/2 [==============================] - ETA: 0s - loss: 0.2496 - accuracy: 0.53 - 1s 494ms/step - loss: 0.2469 - accuracy: 0.5469 - val_loss: 0.2517 - val_accuracy: 0.4380
Epoch 6/250
2/2 [==============================] - ETA: 0s - loss: 0.2505 - accuracy: 0.

Epoch 46/250
2/2 [==============================] - ETA: 0s - loss: 0.2572 - accuracy: 0.40 - 1s 379ms/step - loss: 0.2543 - accuracy: 0.3906 - val_loss: 0.2585 - val_accuracy: 0.4370
Epoch 47/250
2/2 [==============================] - ETA: 0s - loss: 0.2533 - accuracy: 0.50 - 1s 371ms/step - loss: 0.2507 - accuracy: 0.4688 - val_loss: 0.2525 - val_accuracy: 0.4380
Epoch 48/250
2/2 [==============================] - ETA: 0s - loss: 0.2447 - accuracy: 0.65 - 1s 308ms/step - loss: 0.2468 - accuracy: 0.6250 - val_loss: 0.2513 - val_accuracy: 0.4375
Epoch 49/250
2/2 [==============================] - ETA: 0s - loss: 0.2502 - accuracy: 0.53 - 1s 398ms/step - loss: 0.2501 - accuracy: 0.5000 - val_loss: 0.2576 - val_accuracy: 0.4370
Epoch 50/250
2/2 [==============================] - ETA: 0s - loss: 0.2526 - accuracy: 0.43 - 1s 393ms/step - loss: 0.2498 - accuracy: 0.5312 - val_loss: 0.2533 - val_accuracy: 0.4380
Epoch 51/250
2/2 [==============================] - ETA: 0s - loss: 0.2526 - acc

In [62]:
# Evaluating the model
test_loss, test_acc = model.evaluate_generator(test_gen, steps=3)
print('test acc:', test_acc)
print("test_loss:", test_loss)

test acc: 0.5833333134651184
test_loss: 0.24848084151744843


## Without sentiment

In [63]:
#normalizing data
normalized_df1 = normalized_df2

mean = normalized_df1.mean(axis = 0)
normalized_df1 -= mean
std = normalized_df1.std(axis=0)
normalized_df1 /= std

#adding label: up/down or steady
def add_label(df):
    idx = len(df.columns)
    new_col = np.where(df['Close'] >= df['Close'].shift(1), 1, 0)  
    df.insert(loc=idx, column='Label', value=new_col)
    df = df.fillna(0)
    
add_label(normalized_df1)

In [64]:
#applying function 
del normalized_df1['compound_mean']

In [65]:
normalized_df1

,Open,High,Low,Close,Label
0,-1.233622,-1.220002,-1.207656,-1.197743,0
1,-1.181476,-1.169168,-1.167312,-1.153690,1
2,-1.176461,-1.159201,-1.152183,-1.138672,1
3,-1.168439,-1.177142,-1.154200,-1.167707,0
4,-1.175459,-1.178139,-1.169329,-1.176718,0
...,...,...,...,...,...
747,2.148868,2.190869,2.181270,2.195363,0
748,2.222073,2.177911,2.205476,2.168330,0
749,2.429656,2.472948,2.451577,2.471697,1
750,2.548990,2.510825,2.531257,2.527765,1


In [66]:
normalized_df1 = normalized_df1.values

In [67]:
#Splitting into tain validation and test set

train_gen = generator(normalized_df1,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=round(0.6*len(normalized_df1)),
                      shuffle=False,
                      step=step,
                      batch_size=batch_size)
val_gen = generator(normalized_df1,
                    lookback=lookback,
                    delay=delay,
                    min_index=round(0.6*len(normalized_df1))+1,
                    max_index=round(0.8*len(normalized_df1)),
                    step=step,
                    batch_size=batch_size)
test_gen = generator(normalized_df1,
                     lookback=lookback,
                     delay=delay,
                     min_index=round(0.8*len(normalized_df1))+1,
                     max_index=None,
                     step=step,
                     batch_size=batch_size)

val_steps = (round(0.8*len(normalized_df1)) - round(0.6*len(normalized_df1))+1 - lookback) # how many steps to draw from val_gen in order to see the entire validation set
test_steps = (len(normalized_df1) - round(0.8*len(normalized_df1))+1 - lookback)
# How many steps to draw from test_gen in order to see the entire test set

In [68]:
#LSTM model
model = Sequential()
model.add(LSTM(100, return_sequences=True,
                    input_shape=(None, normalized_df1.shape[-1]),
                    kernel_initializer='random_uniform'))
model.add(Dropout(0.4))
model.add(LSTM(60, dropout=0.0, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(20,activation='relu'))
model.add(layers.Dense(2, activation='softmax'))
model.compile(loss='mean_squared_error', optimizer=RMSprop(),metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                       patience=50, min_delta=0.0001, restore_best_weights = True)

history = model.fit_generator(train_gen,
                              steps_per_epoch=2,
                              epochs=200,
                              validation_data=val_gen,
                              validation_steps=val_steps,
                              callbacks=[es])

Epoch 1/200
2/2 [==============================] - ETA: 2s - loss: 0.2512 - accuracy: 0.37 - 3s 2s/step - loss: 0.2507 - accuracy: 0.4375 - val_loss: 0.2504 - val_accuracy: 0.4380
Epoch 2/200
2/2 [==============================] - ETA: 0s - loss: 0.2498 - accuracy: 0.50 - 0s 239ms/step - loss: 0.2507 - accuracy: 0.4375 - val_loss: 0.2500 - val_accuracy: 0.4270
Epoch 3/200
2/2 [==============================] - ETA: 0s - loss: 0.2497 - accuracy: 0.50 - 1s 313ms/step - loss: 0.2497 - accuracy: 0.4844 - val_loss: 0.2495 - val_accuracy: 0.5630
Epoch 4/200
2/2 [==============================] - ETA: 0s - loss: 0.2502 - accuracy: 0.53 - 1s 281ms/step - loss: 0.2507 - accuracy: 0.4844 - val_loss: 0.2501 - val_accuracy: 0.4585
Epoch 5/200
2/2 [==============================] - ETA: 0s - loss: 0.2496 - accuracy: 0.53 - 1s 333ms/step - loss: 0.2499 - accuracy: 0.4688 - val_loss: 0.2500 - val_accuracy: 0.4375
Epoch 6/200
2/2 [==============================] - ETA: 0s - loss: 0.2501 - accuracy: 0.

Epoch 46/200
2/2 [==============================] - ETA: 0s - loss: 0.2503 - accuracy: 0.50 - 1s 364ms/step - loss: 0.2465 - accuracy: 0.5469 - val_loss: 0.2508 - val_accuracy: 0.4380
Epoch 47/200
2/2 [==============================] - ETA: 0s - loss: 0.2468 - accuracy: 0.56 - 1s 302ms/step - loss: 0.2472 - accuracy: 0.5312 - val_loss: 0.2501 - val_accuracy: 0.4375
Epoch 48/200
2/2 [==============================] - ETA: 0s - loss: 0.2540 - accuracy: 0.37 - 1s 347ms/step - loss: 0.2523 - accuracy: 0.4688 - val_loss: 0.2513 - val_accuracy: 0.4370
Epoch 49/200
2/2 [==============================] - ETA: 0s - loss: 0.2530 - accuracy: 0.50 - 1s 284ms/step - loss: 0.2514 - accuracy: 0.5000 - val_loss: 0.2503 - val_accuracy: 0.4380
Epoch 50/200
2/2 [==============================] - ETA: 0s - loss: 0.2451 - accuracy: 0.68 - 1s 358ms/step - loss: 0.2479 - accuracy: 0.5938 - val_loss: 0.2500 - val_accuracy: 0.4375
Epoch 51/200
2/2 [==============================] - ETA: 0s - loss: 0.2482 - acc

In [69]:
#evaluating the model
test_loss, test_acc = model.evaluate_generator(test_gen, steps=4)
print('test acc:', test_acc)
print("test_loss:", test_loss)

test acc: 0.40625
test_loss: 0.25044193863868713


In [70]:
#GRU model
model = Sequential()
model.add(layers.GRU(32,
                     dropout=0.3,
                     recurrent_dropout=0.2,
                     return_sequences=True,
                     input_shape=(None, normalized_df1.shape[-1])))
model.add(layers.GRU(64, activation='relu',
                     dropout=0.3,
                     recurrent_dropout=0.2))
model.add(layers.Dense(2, activation='softmax'))
model.compile(optimizer=RMSprop(), loss='mean_squared_error', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                       patience=100, min_delta=0.0001, restore_best_weights = True)
    
history = model.fit_generator(train_gen,
                              steps_per_epoch=2,
                              epochs=500,
                              validation_data=val_gen,
                              validation_steps=val_steps,
                              callbacks=[es])

Epoch 1/500
2/2 [==============================] - ETA: 2s - loss: 0.2716 - accuracy: 0.37 - 4s 2s/step - loss: 0.2624 - accuracy: 0.4219 - val_loss: 0.2478 - val_accuracy: 0.5620
Epoch 2/500
2/2 [==============================] - ETA: 0s - loss: 0.2514 - accuracy: 0.46 - 1s 314ms/step - loss: 0.2524 - accuracy: 0.4375 - val_loss: 0.2502 - val_accuracy: 0.5625
Epoch 3/500
2/2 [==============================] - ETA: 0s - loss: 0.2517 - accuracy: 0.53 - 1s 267ms/step - loss: 0.2504 - accuracy: 0.5625 - val_loss: 0.2458 - val_accuracy: 0.5630
Epoch 4/500
2/2 [==============================] - ETA: 0s - loss: 0.2521 - accuracy: 0.50 - 1s 344ms/step - loss: 0.2574 - accuracy: 0.4531 - val_loss: 0.2477 - val_accuracy: 0.5620
Epoch 5/500
2/2 [==============================] - ETA: 0s - loss: 0.2531 - accuracy: 0.40 - 1s 287ms/step - loss: 0.2511 - accuracy: 0.4688 - val_loss: 0.2502 - val_accuracy: 0.5625
Epoch 6/500
2/2 [==============================] - ETA: 0s - loss: 0.2536 - accuracy: 0.

Epoch 46/500
2/2 [==============================] - ETA: 0s - loss: 0.2484 - accuracy: 0.43 - 1s 334ms/step - loss: 0.2413 - accuracy: 0.5312 - val_loss: 0.2476 - val_accuracy: 0.5620
Epoch 47/500
2/2 [==============================] - ETA: 0s - loss: 0.2579 - accuracy: 0.40 - 1s 261ms/step - loss: 0.2551 - accuracy: 0.4375 - val_loss: 0.2504 - val_accuracy: 0.5625
Epoch 48/500
2/2 [==============================] - ETA: 0s - loss: 0.2544 - accuracy: 0.37 - 1s 313ms/step - loss: 0.2514 - accuracy: 0.4375 - val_loss: 0.2442 - val_accuracy: 0.5630
Epoch 49/500
2/2 [==============================] - ETA: 0s - loss: 0.2530 - accuracy: 0.46 - 1s 257ms/step - loss: 0.2499 - accuracy: 0.5625 - val_loss: 0.2478 - val_accuracy: 0.5620
Epoch 50/500
2/2 [==============================] - ETA: 0s - loss: 0.2570 - accuracy: 0.53 - 1s 351ms/step - loss: 0.2568 - accuracy: 0.5156 - val_loss: 0.2508 - val_accuracy: 0.5625
Epoch 51/500
2/2 [==============================] - ETA: 0s - loss: 0.2540 - acc

Epoch 91/500
2/2 [==============================] - ETA: 0s - loss: 0.2508 - accuracy: 0.53 - 1s 356ms/step - loss: 0.2485 - accuracy: 0.5312 - val_loss: 0.2476 - val_accuracy: 0.5620
Epoch 92/500
2/2 [==============================] - ETA: 0s - loss: 0.2438 - accuracy: 0.50 - 1s 314ms/step - loss: 0.2477 - accuracy: 0.4844 - val_loss: 0.2499 - val_accuracy: 0.5625
Epoch 93/500
2/2 [==============================] - ETA: 0s - loss: 0.2507 - accuracy: 0.56 - 1s 365ms/step - loss: 0.2537 - accuracy: 0.5000 - val_loss: 0.2445 - val_accuracy: 0.5630
Epoch 94/500
2/2 [==============================] - ETA: 0s - loss: 0.2505 - accuracy: 0.59 - 1s 301ms/step - loss: 0.2522 - accuracy: 0.5000 - val_loss: 0.2474 - val_accuracy: 0.5620
Epoch 95/500
2/2 [==============================] - ETA: 0s - loss: 0.2567 - accuracy: 0.50 - 1s 322ms/step - loss: 0.2513 - accuracy: 0.5156 - val_loss: 0.2499 - val_accuracy: 0.5625
Epoch 96/500
2/2 [==============================] - ETA: 0s - loss: 0.2508 - acc

2/2 [==============================] - ETA: 0s - loss: 0.2578 - accuracy: 0.40 - 1s 490ms/step - loss: 0.2517 - accuracy: 0.4688 - val_loss: 0.2499 - val_accuracy: 0.5730
Epoch 180/500
2/2 [==============================] - ETA: 0s - loss: 0.2541 - accuracy: 0.53 - 1s 405ms/step - loss: 0.2544 - accuracy: 0.4688 - val_loss: 0.2442 - val_accuracy: 0.5630
Epoch 181/500
2/2 [==============================] - ETA: 0s - loss: 0.2488 - accuracy: 0.50 - 1s 460ms/step - loss: 0.2483 - accuracy: 0.5625 - val_loss: 0.2473 - val_accuracy: 0.5620
Epoch 182/500
2/2 [==============================] - ETA: 0s - loss: 0.2484 - accuracy: 0.65 - 1s 350ms/step - loss: 0.2488 - accuracy: 0.6094 - val_loss: 0.2499 - val_accuracy: 0.5415
Epoch 183/500
2/2 [==============================] - ETA: 0s - loss: 0.2405 - accuracy: 0.56 - 1s 287ms/step - loss: 0.2428 - accuracy: 0.5469 - val_loss: 0.2455 - val_accuracy: 0.5323
Epoch 184/500
2/2 [==============================] - ETA: 0s - loss: 0.2494 - accuracy: 0

In [71]:
#Evaluating the model 
test_loss, test_acc = model.evaluate_generator(test_gen, steps=4)
print('test acc:', test_acc)
print("test_loss:", test_loss)

test acc: 0.40625
test_loss: 0.26183515787124634
